In [1]:
import sys
import numpy as np
import pandas as pd
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import nltk
import re
#nltk.download('words')

In [4]:
data = pd.read_csv('Tweets_bis_Apr4.csv')
data.tail()
data.shape



(102994, 8)

In [5]:
text = ''
for i in range(10360):
    text += data.text[i]
    text += '\n'

In [7]:
text = text.lower()

In [10]:
words = set(nltk.corpus.words.words())
text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in words or not w.isalpha())



In [11]:
del data
gc.collect()

147613

In [12]:
import re
text = re.sub(r'\W+', ' ', text)

In [13]:
vocab = sorted(list(set(text)))
len(vocab)

42

In [14]:
char2id = dict((c, i) for i, c in enumerate(vocab))

input_len = len(text)
vocab_len = len(vocab)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 298092
Total vocab: 42


In [15]:
seq_length = 100
X_data = []
y_data = []


In [16]:
for i in range(0, input_len - seq_length, 1):
    in_seq = text[i:i + seq_length]
    out_seq = text[i + seq_length]
    X_data.append([char2id[char] for char in in_seq])
    y_data.append(char2id[out_seq])

In [17]:
n_patterns = len(X_data)
print ("Total Patterns:", n_patterns)

X = np.reshape(X_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

Total Patterns: 297992


In [18]:
from keras.utils import np_utils

y = np_utils.to_categorical(y_data)

In [19]:
gc.collect()

0

In [23]:
training = True

def create_model():
    model = Sequential()
    #model.add(Embedding(vocab_len,64))
    model.add(LSTM (256,return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM (256))
    #model.add(Dropout(0.25))
    #model.add(Dense(64,activation='relu'))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

if training:
    model = create_model()
    model.fit(X, y, epochs=50, batch_size=1024)
    model.save('politiker_tweet.h5')
    '''Pkl_Filename = "Tweeter.pkl"  

    with open(Pkl_Filename, 'wb') as file:  
        pickle.dump(model, file)'''
else:
    from tensorflow import keras
    model = create_model()
    model.built = True
    model.load_weights('politiker_tweet.h5')

Epoch 1/50
 74/292 [======>.......................] - ETA: 40:11 - loss: 3.0937

KeyboardInterrupt: 

In [21]:
import gc
gc.collect()

0

OSError: Unable to open file (unable to open file: name = 'politiker_tweet.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)